In [3]:
#PAQUETES 
import numpy as np
import pandas as pd
from pathlib import Path
import re
import os

# tweeter processing
# Import all needed libraries
import tweepy                   # Python wrapper around Twitter API
import json
import csv
from datetime import date
from datetime import datetime
import time
import webbrowser
import string

In [52]:
base_dir = Path.cwd()
data_dir = base_dir / 'data'
#Create if they don't exist
Path(data_dir).mkdir(parents=True, exist_ok=True)

In [4]:
consumer_key = 'eIVPmXDdEYEtvfJaPLtJD12iV'
consumer_secret = 'aOnHd3rwXOgL2hsmuOXVtGP51NBePNHmovGDRGnKGQSwN6iMQZ'

access_token = '1386719422971355137-pd3S7pBWRLEjXwzI1qRz67Sz9txP9I'
access_token_secret = 'qYqySGmFSJGNVlZYSvr02kjyMmxMWOuORKCKglL5T3C5p'

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [211]:
def get_query(query):
    # initialize a list to hold all the Tweets
    alltweets = []
    data = pd.DataFrame(columns=['tweet_id', 'user_id', 'name', 'screen_name', 'followers_count', 'retweet_count', 'likes_count', 'text', 'mined_at', 'created_at', 'favourite_count', 'retweet_text', 'retweet_user_id', 'retweet_screen_name', 'quote_text', 'quote_user_id', 'quote_screen_name'])
    #name
    print(f'–– {query} –– \n')
    
    #time 
    t0 = time.time()
    
    # make initial request for most recent tweets 
    # (200 is the maximum allowed count)
    new_tweets = api.search(query, result_type='recent', lang='es', count=200, tweet_mode="extended")
    
    # save most recent tweets
    alltweets.extend(new_tweets)
    #print(f"...{len(alltweets)} tweets downloaded so far... {round(time.time() - t0)} s")
    # save the id of the oldest tweet less one to avoid duplication
    
    oldest = alltweets[-1].id - 1
    delta_delta = 0
    # keep grabbing tweets until there are no tweets left
    while len(new_tweets) > 0:
        # all subsequent requests use the max_id param to prevent
        # duplicates
        new_tweets = api.search(query, result_type='recent', lang='es', count=200, max_id=oldest, tweet_mode="extended")
        # save most recent tweets
        alltweets.extend(new_tweets)
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        delta_t = round(time.time() - t0)
        if delta_t >= (delta_delta +10):
            delta_delta = delta_t
            print(f"...{len(alltweets)} tweets downloaded so far... {delta_t} s")
        ### END OF WHILE LOOP ###
        
    # transform the tweepy tweets into a dictionary
    for tweet in alltweets:
        mined = {
            'tweet_id':        tweet.id,
            'user_id':         tweet.user.id,
            'name':            tweet.user.name,
            'screen_name':     tweet.user.screen_name,
            'followers_count': tweet.user.followers_count,
            'retweet_count':   tweet.retweet_count,
            'likes_count':     tweet.favorite_count,
            'text':            tweet.full_text,
            'mined_at':        datetime.now(),
            'created_at':      tweet.created_at,
            'favourite_count': tweet.favorite_count
        }

        try:
            mined['retweet_text'] = tweet.retweeted_status.full_text
            mined['retweet_user_id'] = tweet.retweeted_status.user.id
            mined['retweet_screen_name'] = tweet.retweeted_status.user.screen_name
        except:
            mined['retweet_text'] = 'None'
            mined['retweet_user_id'] = 'None'
            mined['retweet_screen_name'] = 'None'
        try:
            mined['quote_text'] = tweet.quoted_status.full_text
            mined['quote_user_id'] = tweet.quoted_status.user.id
            mined['quote_screen_name'] = tweet.quoted_status.user.screen_name
        except:
            mined['quote_text'] = 'None'
            mined['quote_user_id'] = 'None'
            mined['quote_screen_name'] = 'None'
    
        data = data.append(mined, ignore_index=True)
        
    print(f'\n TIEMPO TOTAL: {round(time.time() - t0)} s \n')
    
    return data

In [208]:
afavor_HTS = ['abortolegalya','CausaJustaTV','SaludReproductivaAhora','LaLibertadEsMiCausa','DerechoAlaIVE','micausajusta','CausaJusta','DerechoALaIVE','CausaJustaPorElAborto','LaLibertadEsMiCausa','CorteAbortoSi']
encontra_HTS = ['sialavida','noalaborto','nohayabortoseguro','yosoyprovida','soyprovida','colombiaesprovida','salvemoslas2vidas','abortoinconstitucional','noseráley','noseraley','CorteAbortoNo']
HTS = dict(aliados=afavor_HTS, opositores=encontra_HTS)

In [216]:
def queries_to_df(dicc_queries):
    dfs = []
    for q in dicc_queries:
        query = ['#'+x for x in HTS[q]]
        query = ' OR '.join(query)

        d = get_query(query)
        d['orientation'] = q
        dfs.append(d)
        d =None

    df = pd.concat(dfs)
    df.to_pickle(data_dir / 'tweets.pkl')